In [201]:
import pandas as pd
import numpy as np
import datetime

df = pd.read_csv('input.csv', header=None)
output = []

In [202]:
df.set_axis(['date','id','event'],axis=1,inplace=True)
df['date'] = pd.to_datetime(df['date'])

In [203]:
PURCHASE = 'PURCHASE'
FRAUD_REPORT = 'FRAUD_REPORT'

CAH = dict.fromkeys(['NO_HISTORY','FRAUD_HISTORY',
                           'GOOD_HISTORY', 'UNCONFIRMED_HISTORY'])

for key in CAH:
    CAH[key] = []

for ID in list(set(df['id'])):
    CAH['NO_HISTORY'].append(ID)

In [204]:
IDlist = list(set(df['id']))
df2 = pd.DataFrame(np.zeros(shape=(3, len(IDlist))), columns=IDlist,
                   index=['fraud_history', 'good_history', 'unconfirmed_history'])

In [205]:
datedict = dict.fromkeys(list(df['id']))

for key in datedict:
    datedict[key] = []

In [206]:
def classify(index):
    ID = df.iloc[index].id
    event = df.iloc[index].event
    
    if event == PURCHASE:
        updateDate(index)
        
        if ID in CAH['NO_HISTORY']:
            CAH['NO_HISTORY'].remove(ID)
            CAH['UNCONFIRMED_HISTORY'].append(ID)
            
    elif event == FRAUD_REPORT:
        
        if ID in CAH['NO_HISTORY']:
            CAH['NO_HISTORY'].remove(ID)
            CAH['FRAUD_HISTORY'].append(ID)

        elif ID in CAH['GOOD_HISTORY']:
            CAH['GOOD_HISTORY'].remove(ID)
            if ID not in CAH['FRAUD_HISTORY']:        
                CAH['FRAUD_HISTORY'].append(ID)
            
        elif ID in CAH['UNCONFIRMED_HISTORY']:
            CAH['UNCONFIRMED_HISTORY'].remove(ID)
            if ID not in CAH['FRAUD_HISTORY']:
                CAH['FRAUD_HISTORY'].append(ID)
        
        df2.loc['fraud_history', ID] += 1

    else:
        CAH['NO_HISTORY'].append(ID)

In [207]:
def writeStatus(index):
    ID = df.iloc[index].id
    date = df.iloc[index].date
    
    
    if ID in CAH['NO_HISTORY']:
        output.append([date.date(), ID, 'NO_HISTORY'])
            
    elif ID in CAH['FRAUD_HISTORY']:
        output.append([date.date(), ID, 'FRAUD_HISTORY:', 
                       df2.loc['fraud_history', ID]])


    elif ID in CAH['GOOD_HISTORY']:
        output.append([date.date(), ID, 'FRAUD_HISTORY:', 
                       df2.loc['good_history', ID]])
            
    elif ID in CAH['UNCONFIRMED_HISTORY']:
        output.append([date.date(), ID, 'FRAUD_HISTORY:', 
                       df2.loc['unconfirmed_history', ID]])

In [208]:
def updateDate(index):
    ID = df.iloc[index].id
    currdate = df.iloc[index].date

    if ID not in CAH['FRAUD_HISTORY']:
        if(len(datedict[ID]) > 0):
            for date in datedict[ID]:
                if(currdate - date) > datetime.timedelta(days=90):
                    df2.loc['unconfirmed_history', ID] -= 1
                    df2.loc['good_history', ID] += 1
                    datedict[ID].remove(date)

                    if ID not in CAH['GOOD_HISTORY']:
                        CAH['GOOD_HISTORY'].append(ID)
                    if ID in CAH['UNCONFIRMED_HISTORY']:
                        CAH['UNCONFIRMED_HISTORY'].remove(ID)

    writeStatus(index)
    
    datedict[ID].append(currdate)            
    df2.loc['unconfirmed_history', ID] += 1

In [209]:
for i in range(len(list(df['date']))):
    classify(i)

In [210]:
df_o = pd.DataFrame(output)
df_o.to_csv('output.csv', index=False, header=False)